In [1]:
from tqdm import tqdm_notebook as tqdm

from kp_utils import *

### Имеется список user_id таких пользователей, у которых, возможно, имеются оценки фильмов

In [2]:
users_with_votes_file = 'users_with_votes_new.txt'

In [3]:
# establishing session
s = requests.Session()
s.headers.update({
        'Referer': 'http://www.kinopoisk.ru',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36',
        'Host': "www.kinopoisk.ru",
        'Upgrade-Insecure-Requests': '1',
        'Cookie': "PHPSESSID=4up7eotcg54tiekor5qfoo3t82; yandex_gid=117954; _csrf_csrf_token=xeXWigy2DNDuZs-Ykb1QOsg0tq_Rn4Y1Ea7njBiP6Pc; i=S8KSVmoEyOoErmebWK+pu4QVd5Tw1HUspTo7oFgymGi3U/w8WkR7/NLb/MNUzhFDOBDZy2QbS3S7pF2E3n12Fea7grk=; rheftjdd=rheftjddVal; _ym_uid=1537645362509535515; _ym_d=1537645362; mda=0; desktop_session_key=6c10cd08f5cd06ccd714bfd07f4230b2387298c9c136ec27907d187d966f63ed250eb3761e80522a29687dc60024de453b19c88877cbbdfb3517499260f8e878f5792b4ba9180c875195d3dba2ac83352a432df4423b4a4051e89b0a6bc0ef14; desktop_session_key.sig=m8k15zm3dmjD3o_B1kHkGBLtKBA; yandexuid=1248256091535079220; user-geo-region-id=10716; user-geo-country-id=2; fuid01=5be7557328b7241c.uZ5IYAVqbcQoZ1b7vD9M7U22ki_C1Qug649LqTF9DZMIInAeLsZQiF-YcXNWmBGOXRSfIi1EZtk3Oe6cCx9AdMxsNyHp-nzE0Eo5ZyUUPEKsdJrrqarG0lpVh-XOKku0; kpunk=1; spravka=dD0xNTQxOTM4NDM3O2k9MmEwMjo2Yjg6YzA0OjFlZTowOjE0NjQ6MDpjMzM7dT0xNTQxOTM4NDM3OTMxNTg3MzQwO2g9NDEyOTg0NDZkYmFlMmZlZjkzYjE4ODc0OWU0ZjMzYjU=; user_country=ru; tc=1; mobile=no; noflash=true; _ym_visorc_22663942=b; hideBlocks=0; my_perpages=%7B%2277%22%3A200%2C%2279votes%22%3A200%7D",
        'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,de;q=0.6',
        'Referer' : 'https://www.kinopoisk.ru',
        'Accept'  : "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        'Connection': 'keep-alive'    
    })

In [4]:
out_filename_users  = 'kp_users.csv'
out_filename_movies = 'kp_movies.csv'
out_filename_votes = 'kp_users_votes.csv'

movies = pd.read_csv(out_filename_movies, delimiter=';', header=0)
users = pd.read_csv(out_filename_users, delimiter=';', header=0)
already_processed_users = {int(u[0]) for u in users.values}
already_saved_movies = {int(m[0]) for m in movies.values}

print ('already_processed_users: ', len(already_processed_users))
print ('already_saved_movies: ', len(already_saved_movies))

# with open(out_filename_users, 'w',encoding='utf-8') as f:    
#     f.write('user_id;nickname;gender;country;city;birth_date\n')
    
# with open(out_filename_movies, 'w',encoding='utf-8') as f:    
#     f.write('movie_id;href;name_rus;name_eng;rating;count_votes;duration\n')
    
# with open(out_filename_votes, 'w',encoding='utf-8') as f:
#     f.write('user_id;movie_id;vote;datetime_watched\n')

already_processed_users:  259728
already_saved_movies:  201744


In [5]:
users_with_votes_file

'users_with_votes_new.txt'

In [6]:
print (out_filename_users)
print (out_filename_movies)
print (out_filename_votes)

kp_users.csv
kp_movies.csv
kp_users_votes.csv


### Вычтем тех пользователей, которые уже обработаны

In [7]:
to_process_users = read_users_set(users_with_votes_file) - already_processed_users

In [8]:
len(to_process_users)

859428

In [9]:
base_url

'https://www.kinopoisk.ru'

### Процесс краулинга. Очень очень очень долго! Может запараллелить?
### А нужны-ли нам все-все-все пользователи?

In [ ]:
users_batch_dict = dict()
votes_batch_dict = dict()
while len(to_process_users) != 0:
    i = 0
    for userid in tqdm(list(to_process_users)):
        try:
            s.headers.update({
                    'Referer' : base_url,
                })
            user  = get_user_info(userid, s)
            if user['nickname'] is None:
                counter['user_nickname_is_None'] += 1
                continue
                
            s.headers.update({
                    'Referer' : make_user_url(userid),
                })
            votes = get_user_votes_for_films(userid, s)
            if votes is not None:
#                 {'/film/vam-i-ne-snilos-1980-45660/': {'nameRus': 'Вам и не снилось... (1980)',
#                   'href': '/film/vam-i-ne-snilos-1980-45660/',
#                   'rating': 8.191,
#                   'count_votes': 46370,
#                   'duration': '86 мин.',
#                   'nameEng': '',
#                   'date': '17.11.2015, 20:51',
#                   'vote': 10},
#                   ...
#                 }
                votes_batch_dict[userid] = [d for href, d in votes.items()]
                users_batch_dict[userid] = user
            else:
                counter['votes_is_None'] += 1

        except Exception as e:
#             print ("ERROR while processing userid {}:".format(userid), str(e))
            counter[str(e)] += 1

        if i > 0 and i % 500 == 0:
            write_batch_user_votes(users_batch_dict, votes_batch_dict, 
                                   already_saved_movies,
                                   out_filename_users, out_filename_votes, out_filename_movies)
            print (counter)
            users_batch_dict = dict()
            votes_batch_dict = dict()
            
        i = i + 1
        already_processed_users.add(userid)

    to_process_users = read_users_set(users_with_votes_file) - already_processed_users
    
if len(users_batch_dict) > 0 and len(votes_batch_dict) > 0:
    write_batch_user_votes(users_batch_dict, votes_batch_dict, 
                           already_saved_movies,
                           out_filename_users, out_filename_votes, out_filename_movies)

print (counter)

Counter({'Extracting count_votes: list index out of range': 309, 'Extracting duration: list index out of range': 146, 'user_nickname_is_None': 18, 'day is out of range for month': 2, "('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))": 1, 'Extracting rating: list index out of range': 1})
Counter({'Extracting count_votes: list index out of range': 687, 'Extracting duration: list index out of range': 282, 'user_nickname_is_None': 18, 'day is out of range for month': 2, "('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))": 1, 'Extracting rating: list index out of range': 1})
Counter({'Extracting count_votes: list index out of range': 1019, 'Extracting duration: list index out of range': 376, 'user_nickname_is_None': 19, 'day is out of range for month': 2, "('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))": 1, 'Extracting rating: list index out of range': 1})
Cou